In [1]:
from __future__ import print_function
#from __future__ import division

import json
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
import gzip
from scipy.misc import imsave
import time
import scipy.ndimage

from keras import backend as K
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D, Dropout

from keras.layers import *
from keras.layers.merge import concatenate
from keras.models import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard
from keras import losses

from keras.preprocessing.image import ImageDataGenerator
from hyperas import optim
from hyperas.distributions import choice, uniform
from hyperopt import Trials, STATUS_OK, tpe

Using TensorFlow backend.


In [2]:
def data():
    # image dimension (array)
    img_rows = 512
    img_cols = 512

    batch_size = 8
    
    def preprocessor(input_img, img_rows, img_cols):
        """
        Resize input images to constants sizes
        :param input_img: numpy array of images
        :return: numpy array of preprocessed images
        """
        output_img = np.ndarray((input_img.shape[0], input_img.shape[1], img_rows, img_cols), dtype=np.uint8)

        for i in range(input_img.shape[0]):
            output_img[i, 0] = cv2.resize(input_img[i, 0], (img_cols, img_rows), interpolation=cv2.INTER_CUBIC)
        return output_img


    def load_data(path_img, path_mask, img_rows, img_cols):
        """
        Load data from project path
        :return: [X, y] numpy arrays containing the [training, validation, test] data and their respective masks.
        """
        print("\nLoading data...\n")
        X = np.load(path_img)
        y = np.load(path_mask)

        X = preprocessor(X, img_rows, img_cols)
        y = preprocessor(y, img_rows, img_cols)

        X = X.astype('float32')

        mean = np.mean(X)  # mean for data centering
        std = np.std(X)  # std for data normalization

        X -= mean
        X /= std

        y = y.astype('float32')
        y /= 255.  # scale masks to [0, 1]
        return X, y
    
    data_path = '/home/malub_local/data/'

    path_img_train = data_path + 'images_69.npy'
    path_mask_train = data_path + 'masks_69.npy'

    path_img_valid = data_path + 'images_valid_29.npy'
    path_mask_valid = data_path + 'masks_valid_29.npy'

    path_img_test = data_path + 'images_test_5.npy'
    path_mask_test= data_path + 'masks_test_5.npy'

    
    X_train, y_train = load_data(path_img_train, path_mask_train, img_rows, img_cols )
    X_valid, y_valid = load_data(path_img_valid, path_mask_valid, img_rows, img_cols)
    return X_train, y_train, X_valid, y_valid

In [3]:
dropout = True
learning_rate = 1e-3
nb_initial_epochs = 10
decay_rate = learning_rate / nb_initial_epochs


def create_model(X_train, y_train, X_valid, y_valid):
    """
    Model providing function:

    Create Keras model with double curly brackets dropped-in as needed.
    Return value has to be a valid python dictionary with two customary keys:
        - loss: Specify a numeric evaluation metric to be minimized
        - status: Just use STATUS_OK and see hyperopt documentation if not feasible
    The last one is optional, though recommended, namely:
        - model: specify the model just created so that we can later use it again.
        
    """
    K.set_image_dim_ordering('th') 
    dropout = True
    learning_rate = 1e-3
    nb_initial_epochs = 10
    decay_rate = learning_rate / nb_initial_epochs
    
    #### loss and metrics #####
    def dice_coef(y_true, y_pred, smooth = 1.):
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        intersection = K.sum(y_true_f * y_pred_f)
        return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


    """def dice_coef(y_true, y_pred, smooth=1):

        intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
        return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)"""



    def weighted_binary_crossentropy(y_true, y_pred):
            # Calculate the binary crossentropy
        b_ce = K.binary_crossentropy(y_true, y_pred)

            # Apply the weights
        weight_vector = y_true * 0.30 + (1. - y_true) * 0.70
        weighted_b_ce = weight_vector * b_ce

            # Return the mean error
        return K.mean(weighted_b_ce)

    
    inputs = Input((1, img_rows, img_cols))
    #conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(inputs)
    conv1 = Conv2D(32, (3, 3), padding="same", activation="relu")(inputs)
    conv1 = Conv2D(32, (3, 3), padding="same", activation="relu")(conv1)
    batch1 = BatchNormalization(axis=1)(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(batch1)


    conv2 = Conv2D(64, (3, 3), padding="same", activation="relu")(pool1)
    conv2 = Conv2D(64, (3, 3), padding="same", activation="relu")(conv2)
    batch2 = BatchNormalization(axis=1)(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(batch2)

   
    conv3 = Conv2D(128, (3, 3), padding="same", activation="relu")(pool2)
    conv3 = Conv2D(128, (3, 3), padding="same", activation="relu")(conv3)
    batch3 = BatchNormalization(axis=1)(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(batch3)

    conv4 = Conv2D(256, (3, 3), padding="same", activation="relu")(pool3)
    conv4 = Conv2D(256, (3, 3), padding="same", activation="relu")(conv4)
    batch4 = BatchNormalization(axis=1)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(batch4)
    
    if dropout:
        pool4 = Dropout(0.5)(pool4)
   
    conv5 = Conv2D(512, (3, 3), padding="same", activation="relu")(pool4)
    conv5 = Conv2D(512, (3, 3), padding="same", activation="relu")(conv5)

    if dropout:
        conv5 = Dropout({{choice([0.3, 0.5, 0.7])}})(conv5)    

    up6_interm = UpSampling2D(size=(2, 2))(conv5)
    
    up6 = concatenate([up6_interm, conv4], axis=1)

    conv6 = Conv2D(256, (3, 3), padding="same", activation="relu")(up6)
    conv6 = Conv2D(256, (3, 3), padding="same", activation="relu")(conv6)
    batch6 = BatchNormalization(axis=1)(conv6)

    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis=1)
    
    if dropout:
        up7 = Dropout({{choice([0.3, 0.5])}})(up7)  
        
    conv7 = Conv2D(128, (3, 3), padding="same", activation="relu")(up7)
    conv7 = Conv2D(128, (3, 3), padding="same", activation="relu")(conv7)
    batch7 = BatchNormalization(axis=1)(conv7)

    up8 = concatenate([UpSampling2D(size=(2, 2))(batch7), conv2], axis=1)

    conv8 = Conv2D(64, (3, 3), padding="same", activation="relu")(up8)
    conv8 = Conv2D(64, (3, 3), padding="same", activation="relu")(conv8)
    batch8 = BatchNormalization(axis=1)(conv8)

    up9 = concatenate([UpSampling2D(size=(2, 2))(batch8), conv1], axis=1)

    conv9 = Conv2D(32, (3, 3), padding="same", activation="relu")(up9)
    conv9 = Conv2D(32, (3, 3), padding="same", activation="relu")(conv9)
    batch9 = BatchNormalization(axis=1)(conv9)

    conv10 = Conv2D(1, (1, 1), activation="sigmoid")(batch9)

    model = Model(outputs=conv10, inputs=inputs)

    model.compile(optimizer=Adam(lr = learning_rate, decay=decay_rate), loss=weighted_binary_crossentropy, 
                  metrics = [dice_coef])
    
    
    model.fit(X_train, y_train, 
                        validation_data = (X_valid, y_valid),
                          batch_size = 8,
                        epochs = 10, 
                        verbose = 1,
                        shuffle = True
                        ) 
                                  #validation_steps= 5,
                                  #callbacks = [tensorboard])


    score, dice = model.evaluate(X_valid, y_valid, verbose=0)
    print('Test accuracy:', dice)
    return {'loss': -dice, 'status': STATUS_OK, 'model': model}




In [4]:
if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=5,
                                          trials=Trials(),
                                          notebook_name='hyperparam_tuning')
    X_train, y_train, X_valid, y_valid = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_valid, y_valid))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import json
except:
    pass

try:
    import os
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import cv2
except:
    pass

try:
    import gzip
except:
    pass

try:
    from scipy.misc import imsave
except:
    pass

try:
    import time
except:
    pass

try:
    import scipy.ndimage
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D, Dropout
except:
    pass

try:
    from keras.layers import *
except:
    pass

try:
    from keras.layers.merge import concatenate
except:
    pass

try:
    from keras.models import *
except:
    pass

try:
    from keras.optimizers import *
except:
    pass

try:
    from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard
except:
    pass

try:
    from

Train on 69 samples, validate on 29 samples
Epoch 1/10
69/69 [==============================] - 9s 129ms/step - loss: 0.3359 - dice_coef: 0.5639 - val_loss: 1.4112 - val_dice_coef: 0.0228
Epoch 2/10
69/69 [==============================] - 4s 59ms/step - loss: 0.2438 - dice_coef: 0.6503 - val_loss: 0.3842 - val_dice_coef: 0.3564
Epoch 3/10
69/69 [==============================] - 4s 60ms/step - loss: 0.2285 - dice_coef: 0.6578 - val_loss: 0.2146 - val_dice_coef: 0.4795
Epoch 4/10
69/69 [==============================] - 4s 60ms/step - loss: 0.2138 - dice_coef: 0.6545 - val_loss: 0.2489 - val_dice_coef: 0.3677
Epoch 5/10
69/69 [==============================] - 4s 60ms/step - loss: 0.2048 - dice_coef: 0.6625 - val_loss: 0.1966 - val_dice_coef: 0.4255
Epoch 6/10
69/69 [==============================] - 4s 60ms/step - loss: 0.1969 - dice_coef: 0.6588 - val_loss: 0.2068 - val_dice_coef: 0.3567
Epoch 7/10
69/69 [==============================] - 4s 59ms/step - loss: 0.1948 - dice_coef: 0.65